In [1]:
#%%% JSON content script
import pandas as pd
import json
from sqlalchemy import create_engine

# Make a connection to the database of Netivity / Optimizers using SQLAlchemy
connection_string = 'mssql+pyodbc:///?odbc_connect=' \
                    'Driver%3D%7BODBC+Driver+17+for+SQL+Server%7D%3B' \
                    'Server%3Dsql-maxaro-prod.database.windows.net%3B' \
                    'Database%3Dsqldb-maxaro-prod%3B' \
                    'UID%3Dbiadmin@maxaro.nl%3B' \
                    'Authentication%3DActiveDirectoryInteractive'

engine = create_engine(connection_string)

# Create the SQL-query to retrieve the "EmailID," "To," and "Content" columns from the "Emails" table
sql = """SELECT JsonContent, EmailContentType, [To]
         FROM Emails
         WHERE EmailContentType IN (
            'NetivityEcommerce.Core.Domain.Emails.Contents.ServiceForms.IncorrectDelivery.ServiceFormIncorrectDeliveryEmailContent',
            'NetivityEcommerce.Core.Domain.Emails.Contents.ServiceForms.Installation.ServiceFormInstallationEmailContent',
            'NetivityEcommerce.Core.Domain.Emails.Contents.ServiceForms.Defect.ServiceFormDefectEmailContent'
        );"""

# Get the email data as a Pandas DataFrame
json_data = pd.read_sql(sql, engine)

# Close the database connection
engine.dispose()


C:\Users\bgraziadei\AppData\Local\Temp\ipykernel_19608\498525398.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  json_data = pd.read_sql(sql, engine)


AttributeError: 'Engine' object has no attribute 'cursor'

In [2]:
data = pd.DataFrame(json_data)

def is_valid_json(jstr):
    try:
        json.loads(jstr)
        return True
    except:
        return False

# Keep only valid JSON rows
valid_data = data[data['JsonContent'].apply(is_valid_json)].copy()

# Convert the 'JsonContent' string to a dictionary
valid_data['JsonContentDict'] = valid_data['JsonContent'].apply(json.loads)

# Extract the info from each dictionary
valid_data['ArticleNumber'] = valid_data['JsonContentDict'].apply(lambda x: x['Products'][0]['ArticleNumber'] if 'Products' in x and x['Products'] else None)
valid_data['ProductIsAssembled'] = valid_data['JsonContentDict'].apply(lambda x: x['Products'][0]['ProductIsAssembled'] if 'ProductIsAssembled' in x['Products'][0] else None)
valid_data['Comment'] = valid_data['JsonContentDict'].apply(lambda x: x['Products'][0]['Comment'] if 'Comment' in x['Products'][0] else None)

# Extract 'OrderNumber' from each dictionary and remove any '#' from the start
valid_data['OrderNumber'] = valid_data['JsonContentDict'].apply(lambda x: x.get('OrderNumber', None)).str.lstrip('#')
for number in valid_data['OrderNumber']:
    number.upper()

# pattern for how an article number should look like
article_pattern = r'(A\d{7})'
valid_data['Type'] = 'Search'  # default all to Search and then change the ones that are Articles

# Classify 'ArticleNumber'
valid_data['ITEMNUMBER'] = valid_data['ArticleNumber'].str.extract(article_pattern)
valid_data.loc[valid_data['ArticleNumber'].str.match(article_pattern, na=False), 'Type'] = 'Article'
valid_data['searchData'] = valid_data.loc[valid_data['Type'] == 'Search', 'ArticleNumber']

# Classify 'OrderNumber' for both MNHQ and Others
mnhq_pattern = r'^MNHQ'
other_order_pattern = r'^(IMX|WMX|BMX)'
valid_data['MNHQ_Order'] = valid_data['OrderNumber'][valid_data['OrderNumber'].str.match(mnhq_pattern, na=False)]
valid_data['Other_Order'] = valid_data.loc[valid_data['OrderNumber'].str.match(other_order_pattern, na=False), 'OrderNumber']

In [3]:
json_data

,JsonContent,EmailContentType,To
0,"{\r\n ""Gender"": ""Women"",\r\n ""FirstName"": ""M...",NetivityEcommerce.Core.Domain.Emails.Contents....,marielouise.lentjes@gmail.com
1,"{\r\n ""Gender"": ""Men"",\r\n ""FirstName"": ""Jef...",NetivityEcommerce.Core.Domain.Emails.Contents....,j+maxaro@drsn.eu
2,"{\r\n ""Gender"": ""Men"",\r\n ""FirstName"": ""Maa...",NetivityEcommerce.Core.Domain.Emails.Contents....,maarten.vangiel@hotmail.com
3,"{\r\n ""Gender"": ""Men"",\r\n ""FirstName"": ""F.""...",NetivityEcommerce.Core.Domain.Emails.Contents....,fvanzwicht@hotmail.com
4,"{\r\n ""Gender"": ""Men"",\r\n ""FirstName"": ""Jer...",NetivityEcommerce.Core.Domain.Emails.Contents....,klusbedrijf.lef@gmail.com
...,...,...,...
683,"{\r\n ""Gender"": ""Men"",\r\n ""FirstName"": ""Mic...",NetivityEcommerce.Core.Domain.Emails.Contents....,michaelstreet1987@hotmail.com
684,"{\r\n ""Gender"": ""Men"",\r\n ""FirstName"": ""Ric...",NetivityEcommerce.Core.Domain.Emails.Contents....,richbarrett84@gmail.com
685,"{\r\n ""Gender"": ""Men"",\r\n ""FirstName"": ""avt...",NetivityEcommerce.Core.Domain.Emails.Contents....,woodpeckerbouw@gmail.com
686,"{\r\n ""Gender"": ""Men"",\r\n ""FirstName"": ""erw...",NetivityEcommerce.Core.Domain.Emails.Contents....,erwindiana@hotmail.com


In [4]:
valid_data

,JsonContent,EmailContentType,To,JsonContentDict,ArticleNumber,ProductIsAssembled,Comment,OrderNumber,Type,ITEMNUMBER,searchData,MNHQ_Order,Other_Order
0,"{\r\n ""Gender"": ""Women"",\r\n ""FirstName"": ""M...",NetivityEcommerce.Core.Domain.Emails.Contents....,marielouise.lentjes@gmail.com,"{'Gender': 'Women', 'FirstName': 'Marie-Louise...",Set:N128-0031 A0001944,True,De houder waar de cilinder van de stop is beve...,MNHQSO0387516,Search,A0001944,Set:N128-0031 A0001944,MNHQSO0387516,NaN
1,"{\r\n ""Gender"": ""Men"",\r\n ""FirstName"": ""Jef...",NetivityEcommerce.Core.Domain.Emails.Contents....,j+maxaro@drsn.eu,"{'Gender': 'Men', 'FirstName': 'Jeffrey', 'Mid...","1 x Mano 01 handgreep, alu, 256 mm",True,Handgreep vertoond beschadigingen (spikkels/bu...,WMX547834,Search,NaN,"1 x Mano 01 handgreep, alu, 256 mm",NaN,WMX547834
2,"{\r\n ""Gender"": ""Men"",\r\n ""FirstName"": ""Maa...",NetivityEcommerce.Core.Domain.Emails.Contents....,maarten.vangiel@hotmail.com,"{'Gender': 'Men', 'FirstName': 'Maarten', 'Mid...",A0013119,False,We hebben na ontvangst alle tegels geïnspectee...,PS483141,Article,A0013119,NaN,NaN,NaN
3,"{\r\n ""Gender"": ""Men"",\r\n ""FirstName"": ""F.""...",NetivityEcommerce.Core.Domain.Emails.Contents....,fvanzwicht@hotmail.com,"{'Gender': 'Men', 'FirstName': 'F.', 'MiddleNa...",A0012536 - SP08C. Muurprofiel voor F49P,False,Het betreft een aparte levering waarbij 2 glas...,Factuurnummer S1286312. Klantnummer C000243080,Article,A0012536,NaN,NaN,NaN
4,"{\r\n ""Gender"": ""Men"",\r\n ""FirstName"": ""Jer...",NetivityEcommerce.Core.Domain.Emails.Contents....,klusbedrijf.lef@gmail.com,"{'Gender': 'Men', 'FirstName': 'Jeroen', 'Midd...",XZ03-1880XLSM,True,Wegens andere keuzes in de verbouwing is er ne...,MNHQSO0251892,Search,NaN,XZ03-1880XLSM,MNHQSO0251892,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
683,"{\r\n ""Gender"": ""Men"",\r\n ""FirstName"": ""Mic...",NetivityEcommerce.Core.Domain.Emails.Contents....,michaelstreet1987@hotmail.com,"{'Gender': 'Men', 'FirstName': 'Michael', 'Mid...",55.107.300KP,None,None,MNHQSO0365368,Search,NaN,55.107.300KP,MNHQSO0365368,NaN
684,"{\r\n ""Gender"": ""Men"",\r\n ""FirstName"": ""Ric...",NetivityEcommerce.Core.Domain.Emails.Contents....,richbarrett84@gmail.com,"{'Gender': 'Men', 'FirstName': 'Richard', 'Mid...",A0000898,None,None,WMX721102,Article,A0000898,NaN,NaN,WMX721102
685,"{\r\n ""Gender"": ""Men"",\r\n ""FirstName"": ""avt...",NetivityEcommerce.Core.Domain.Emails.Contents....,woodpeckerbouw@gmail.com,"{'Gender': 'Men', 'FirstName': 'avtar ', 'Midd...",BIC55-00361,None,None,WMX757685,Search,NaN,BIC55-00361,NaN,WMX757685
686,"{\r\n ""Gender"": ""Men"",\r\n ""FirstName"": ""erw...",NetivityEcommerce.Core.Domain.Emails.Contents....,erwindiana@hotmail.com,"{'Gender': 'Men', 'FirstName': 'erwin', 'Middl...",Lusso 2 stoomcabine,None,None,CAS0116750,Search,NaN,Lusso 2 stoomcabine,NaN,NaN


In [5]:
#7
article_df = valid_data.loc[valid_data['Type'] == 'Article', ['ITEMNUMBER', 'MNHQ_Order', 'Other_Order', 'Comment', 'ProductIsAssembled', 'To', 'EmailContentType']]#.to_excel("C:\\Users\\bgraziadei\\OneDrive - Maxaro\\Documenten\\JsonContentarticle.xlsx", index=False)
search_df = valid_data.loc[valid_data['Type'] == 'Search', ['searchData', 'MNHQ_Order', 'Other_Order', 'Comment', 'ProductIsAssembled', 'To', 'EmailContentType']]#.to_excel("C:\\Users\\bgraziadei\\OneDrive - Maxaro\\Documenten\\JsonContentsearch.xlsx", index=False)

In [6]:
# Make a connection to the database of Netivity / Optimizers using SQLAlchemy
connection_string = 'mssql+pyodbc:///?odbc_connect=' \
                    'Driver%3D%7BODBC+Driver+17+for+SQL+Server%7D%3B' \
                    'Server%3Dmaxreportsrvr.database.windows.net%3B' \
                    'Database%3Dmax_report_db%3B' \
                    'UID%3Dreportadmin%3B' \
                    'PWD%3D#DAff!%nz8r7'\

engine = create_engine(connection_string)

# Create the SQL-query to retrieve the "EmailID," "To," and "Content" columns from the "Emails" table
sql = """SELECT T.SALESORDERNUMBER, T.CUSTOMERREQUISITIONNUMBER, T.MAXWEBINTEGRATIONSETCODEID, T.ITEMNUMBER
        FROM SalesOrderLineV2Staging T
        INNER JOIN SalesOrderHeaderV2Staging S ON S.SALESORDERNUMBER = T.SALESORDERNUMBER
        WHERE T.DATAAREAID = 'MNHQ';"""
# Get the order data as a Pandas DataFrame
order_data = pd.read_sql(sql, engine)

# Close the database connection
engine.dispose()

In [7]:
# Make a connection to the database of Netivity / Optimizers using SQLAlchemy
connection_string = 'mssql+pyodbc:///?odbc_connect=' \
                    'Driver%3D%7BODBC+Driver+17+for+SQL+Server%7D%3B' \
                    'Server%3Dmaxreportsrvr.database.windows.net%3B' \
                    'Database%3Dmax_report_db%3B' \
                    'UID%3Dreportadmin%3B' \
                    'PWD%3D#DAff!%nz8r7'\

engine = create_engine(connection_string)

# Create the SQL-query to retrieve the "EmailID," "To," and "Content" columns from the "Emails" table
sql = """SELECT T.Productnumber, T.ProductSearchname FROM [dbo].[EcoResProductV2Staging] T"""
# Get the order data as a Pandas DataFrame
product_data = pd.read_sql(sql, engine)

# Close the database connection
engine.dispose()

In [8]:
search_df = search_df.merge(product_data, how='inner', left_on='searchData', right_on='ProductSearchname')
search_df.rename(columns={'Productnumber': 'ITEMNUMBER'}, inplace=True)

In [9]:
article_df.dropna(subset=['MNHQ_Order', 'Other_Order'], how='all', inplace=True)
merge_key1 = article_df.merge(order_data, left_on = ['MNHQ_Order', 'ITEMNUMBER'], right_on = ['SALESORDERNUMBER', 'ITEMNUMBER'], how = 'left')         
merge_key2 = article_df.merge(order_data, left_on = ['Other_Order', 'ITEMNUMBER'], right_on = ['CUSTOMERREQUISITIONNUMBER', 'ITEMNUMBER'], how = 'left')
result_art = pd.concat([merge_key1, merge_key2], ignore_index=True)

In [10]:
search_df.dropna(subset=['MNHQ_Order', 'Other_Order'], how='all', inplace=True)
merge_key1 = search_df.merge(order_data, left_on = ['MNHQ_Order', 'ITEMNUMBER'], right_on = ['SALESORDERNUMBER', 'ITEMNUMBER'], how = 'left')         
merge_key2 = search_df.merge(order_data, left_on = ['Other_Order', 'ITEMNUMBER'], right_on = ['CUSTOMERREQUISITIONNUMBER', 'ITEMNUMBER'], how = 'left')
result_ser = pd.concat([merge_key1, merge_key2], ignore_index=True)

In [11]:
wanted_col = ['ITEMNUMBER', 'Comment', 'ProductIsAssembled', 'To', 'EmailContentType', 'SALESORDERNUMBER', 'CUSTOMERREQUISITIONNUMBER', 'MAXWEBINTEGRATIONSETCODEID']
result = pd.concat([result_art[wanted_col], result_ser[wanted_col]], ignore_index=True)

In [12]:
result.drop_duplicates(inplace=True)

In [13]:
result

,ITEMNUMBER,Comment,ProductIsAssembled,To,EmailContentType,SALESORDERNUMBER,CUSTOMERREQUISITIONNUMBER,MAXWEBINTEGRATIONSETCODEID
0,A0006798,De verwarming lekt water uit bijna alle buizen...,True,guvorsch@hotmail.com,NetivityEcommerce.Core.Domain.Emails.Contents....,MNHQSO0433648,,
2,A0006297,"Beste Maxaro,\n\nBij het omhoog tillen van de ...",False,shelbyperez@hotmail.com,NetivityEcommerce.Core.Domain.Emails.Contents....,NaN,NaN,NaN
3,A0002411,De chromen kleur kraakt af en valt van de grep...,True,info@steiho.nl,NetivityEcommerce.Core.Domain.Emails.Contents....,MNHQSO0269425,WMX613306,Ttjjvq
5,A0004549,Bij vullen van de handdoekradiator bleken meer...,True,annechienverkade@gmail.com,NetivityEcommerce.Core.Domain.Emails.Contents....,MNHQSO0410620,IMX731363,DR57_0517RDN-E
7,A0003049,Een pak met 60x60 tegels is gebroken geleverd....,False,benballegooijen@home.nl,NetivityEcommerce.Core.Domain.Emails.Contents....,MNHQSO0393281,IMX715690,304-020101
...,...,...,...,...,...,...,...,...
997,A0006794,None,None,amysweers@live.nl,NetivityEcommerce.Core.Domain.Emails.Contents....,MNHQSO0405074,IMX726314,DR63_0513RDN
999,A0003702,None,None,roodmatthijs@gmail.com,NetivityEcommerce.Core.Domain.Emails.Contents....,MNHQSO0425463,IMX745278,505-010102
1001,A0001679,None,None,michielmeganck@gmail.com,NetivityEcommerce.Core.Domain.Emails.Contents....,NaN,NaN,NaN
1002,A0001679,None,None,Josvillo@yahoo.es,NetivityEcommerce.Core.Domain.Emails.Contents....,NaN,NaN,NaN


In [14]:
missing_values = result.isna().sum()

print(missing_values)

ITEMNUMBER                      0
Comment                       236
ProductIsAssembled            234
To                              0
EmailContentType                0
SALESORDERNUMBER              344
CUSTOMERREQUISITIONNUMBER     344
MAXWEBINTEGRATIONSETCODEID    344
dtype: int64


In [34]:
result.to_excel(r"M:\ERP\Data\Setcodes\SetCodesMerged.xlsx", index=False)